### Introduction to the data:

In the energy market we work mainly with timeseries data. Most of the data is downloadable from a free or paid source that we access to. However, some of the sources are not reliable, therefore there is a need for data cleaning, e.g. dealing with outliers, missing values, mismatch in date for some of the commodities. There are many ways how to deal with them and we leave it up to you. In the attached files you can find the most important data for our business:

* Power prices: electricity market is one of the pillars of EPH energy business as it represents the revenue side.
* Gas and Coal prices: these are the most common fuels in energy generation worldwide. 
* Emissions prices: in most European countries the power generation is obliged to pay for the emissions produced because of fuel burn. It became an important factor which is making a difference in the profitability of the power generation business. 
* FX: not all commodities are traded in EUR therefore some currency conversion might be needed.

### Tasks:

Try to accomplish as many tasks as possible. You don't have to solve the tasks in the order they are listed but some of them may need the result of previous ones.

In [ ]:
# Load here the packages you will use

import pandas as pd
import sqlite3
import time


1.	Load attached csv files ('commodities', 'fx') into python. Files might contain outliers or missing data that you have to deal with, please describe the logic on how and why you cleaned the data. Specifically, propose a way to replace NaN values and shortly describe the reason behind your choice.

2.	Please calculate for each commodity (for Coal both in USD and EUR) for every calendar year:

 * Average price 
 * Minimum 
 * Maximum
 * Volatility (standard deviation)

3. Using the commodity prices calculate new vectors according to the following equations:
 * CDS = Power price in EUR – 0.37 * coal price in EUR – 0.86 * Emissions in EUR <br>
 (CDS = Clean Dark Spread shows the profitability of a coal power plant, i.e. power - coal variable costs)
 * CSS = Power price in EUR – 2 * gas price in EUR – 0.4 * Emissions in EUR <br>
 (CSS = Clean Spark Spread shows the profitability of a gas power plant, i.e. power - gas variable costs)

4. Show graphically the CDS and CSS in one chart (time on horizontal, price on vertical axis) and represent the difference between the CDS and CSS on the secondary vertical axis.

5. Write an SQL query to make a new table called "profitability" in the provided database file testdatabase.db, consisting of columns 'trade_date', 'cds' and 'css' and fill it with CDS and CSS timeseries obtained in previous step.

In [ ]:
conn = sqlite3.connect("testdatabase.db")
curs = conn.cursor()

table = """ 
        YOUR QUERY
        """

curs.execute(table)
conn.close()

With the table created, compare the historical CSS and CDS values with the past availabilities of some of the company’s power plants. This registry of past availability days already exists in the database in a table called “availability” with the following format:

| trade_date | coal_AX23 | coal_BM12 | gas_N9 |
| --- | --- | --- | --- |
| 24-09-2020 | TRUE | TRUE | TRUE |
| 23-09-2020 | TRUE | TRUE | FALSE |
| 22-09-2020 | TRUE | TRUE | TRUE |
| 21-09-2020 | FALSE | TRUE | TRUE |
| 18-09-2020 | FALSE | TRUE | TRUE |
| ... | ... | ... | ... |

The power plant is able to produce electricity on TRUE days while a maintenance is going on FALSE days, meaning the plant cannot generate power. 

6. Write an SQL query to select the days on which the CSS was positive (greater than zero) and, at the same time, the power plant “gas_N9” was available for production. The result should only include a 'trade_date' column and a 'css' column with the CSS numerical values.

In [ ]:
conn = sqlite3.connect("testdatabase.db")
curs = conn.cursor()

df = pd.read_sql_query("YOUR QUERY", conn)

conn.close()

7. Write an SQL query to count the number of days per calendar quarter (Q1: Jan-Mar, Q2: Apr-Jun etc.) when both “coal_AX23” and “coal_BM12” were available for generation and the CDS was higher than the CSS. The result should include columns with the year, quarter and counted number of days that meet the requirements, for instance:

| year | quarter | count |
| --- | --- | --- |
| 2020 | 4 | 24 |
| 2020 | 3 | 65 |
| 2020 | 2 | 73 |
| 2020 | 1 | 56 |
| 2019 | 4 | 76 |
| 2019 | 3 | 82 |
| ... | ... | ... |

In [ ]:
conn = sqlite3.connect("testdatabase.db")
curs = conn.cursor()

df = pd.read_sql_query("YOUR QUERY", conn)

conn.close()

<br>
<br>
The aim of the next exercise is to simulate the cross-border power transmission from Germany to France. In order to do this, an Excel file containing a dataframe like the one below is provided:

|datetime            | de_price | fr_price | signal |
|---------------------|----------|----------|--------|
| 2013-07-02 00:00:00 | 31.95    | 32.33    | NaN    |
| 2013-07-02 01:00:00 | 30.01    | 30.34    | NaN    |
| 2013-07-02 02:00:00 | 28.30     | 26.23     | NaN    |
| 2013-07-02 03:00:00 | 25.84    | 27.10     | NaN    |
| 2013-07-02 04:00:00 | 26.72    | 28.81    | NaN    |

Note: Data in the dataframe is already correctly treated, there is no need to treat repeated dates or NaN values in the 'price' columns.


Aside of the electricity prices for each country, there is an empty column named "signal" that needs
to be filled with a string saying either 'nominate' or 'do nothing'. The condition is as follows: in each of the hours (rows), if the
French price is greater than the German one and the same thing happened in the previous hour (row above), this hour will be labeled as 'nominate', otherwise it will be labeled as 'do nothing'.

By following these instructions, the sample shown above should have its "signal" column as follows:


|datetime            | de_price | fr_price | signal |
|---------------------|----------|----------|--------|
| 2013-07-02 00:00:00 | 31.95    | 32.33    | do nothing    |
| 2013-07-02 01:00:00 | 30.01    | 30.34    | nominate    |
| 2013-07-02 02:00:00 | 28.30     | 26.23     | do nothing    |
| 2013-07-02 03:00:00 | 25.84    | 27.10     | do nothing    |
| 2013-07-02 04:00:00 | 26.72    | 28.81    | nominate    |

Since the top-row hour is the first one on the dataset, it will be automatically labeled as "do nothing".

8. In order to accomplish this task, a very inefficient chunk of code is given as an example. Modify it or rewrite completely in order to get the same result, but faster:

In [ ]:
power_prices = pd.read_excel('power_prices_history.xlsx', index_col=0)
start = time.time()


####### THE INEFFICIENT CODE STARTS HERE

for i in range(len(power_prices)):
    current_datetime = power_prices.iloc[i].name
    if i==0:
        power_prices.at[current_datetime, 'signal'] = 'do nothing'
    else:
        difference_now = power_prices.iloc[i]['fr_price'] - power_prices.iloc[i]['de_price']
        difference_previous_hour = power_prices.iloc[i-1]['fr_price'] - power_prices.iloc[i-1]['de_price']

        if difference_now > 0 and difference_previous_hour > 0:
            power_prices.at[current_datetime, 'signal'] = 'nominate'
        else:
            power_prices.at[current_datetime, 'signal'] = 'do nothing'

####### THE INEFFICIENT CODE ENDS HERE

            
end = time.time()
print("Seconds:", round(end - start, 2))

display(power_prices)

##### Optional: <br>
9. Write a function according to the instructions in the extended docstring and apply it to the historical Q1 2020 gas prices (obtained after data cleaning from the file 'commodities.csv' in one of the previous steps):

In [ ]:
def gas_max_profit( gas_prices ):
    """
    Arguments: 
        gas_prices: numpy array / list of historical gas prices in chronological order
        
    Output:
        numerical value with the maximum profit found
    
    Instructions:
        * Assume that you are given a perfect price forecast of all future days, i.e. you can use real historical prices.
        * You can only trade a fixed amount of one unit.
        * Prices in the array are listed in chronological order.
        * You must buy before you can sell and must sell before you can buy again.
    
    Examples:
        * Input: [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
          Output: 0
          You will never be able to resell what you buy for a higher price.

        * Input: [1, 2, 3, 4, 5, 6, 7, 8, 9]
          Output: 8
          You would buy on day 1 (price = 1) and sell on day 9 (price = 9).

        * Input: [8, 2, 5, 1, 6, 4]
          Output: 8
          You would buy on day 2 (price = 2), sell on day 3 (profit = 3), buy again on day 4 (price = 1) and sell on day 5 (profit = 5).
    """
    
    # Insert your code here
    